<div style="text-align:right">
    <b>Author:</b> Andreas P. Koenzen (akc at apkc.net) / <a href="http://www.apkc.net">www.apkc.net</a>
</div>

# Perceptron Classifier

### Notes:
1. Dataset can be found __[here](http://www.apkc.net/data)__.
1. See notes by hand __[here](http://www.apkc.net/_8)__.

## Imports

In [ ]:
%run '../../../../imports.py'

## Configuration

In [ ]:
%run '../../../../config.py'

## Environment variables

In [ ]:
%run '../../../../env_variables.py'

## Bankruptcy Example
- L = is the number of late payments on credit cards over the past year.
- R = is the ratio of earnings to expenses (earnings : expenses).
- Bankrupt = **-1 is NO / 1 is YES**.

In [ ]:
data = pd.read_csv("http://www.apkc.net/data/csc_578d/perceptron_1.csv")
data

## Transform Data

In [ ]:
data = pd.DataFrame(
    data={
        0: data['R'],      # X
        1: data['L'],      # Y
        2: data['Bankupt'] # Label
    }
)
data

## Plot the Data (2D)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

_ = ax.scatter(
    x=data[0], 
    y=data[1], 
    c=data[2], 
    cmap=mpl.colors.ListedColormap(['red', 'blue']),
    marker='o', 
    s=40)

_ = ax.set_xlabel('Earning:Expenses')
_ = ax.set_ylabel('Late Payments')

## Plot the Data (3D)

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.gca(projection='3d')

_ = ax.scatter3D(xs=data[0],
                 ys=data[1],
                 zs=data[2],
                 c=data[2],
                 cmap=mpl.colors.ListedColormap(['red', 'blue']),
                 marker='o',
                 s=40)

_ = ax.set_xlabel('Earning:Expenses')
_ = ax.set_ylabel('Late Payments')
_ = ax.set_zlabel('Class')

## Compute the Weights of the Line

Do N iterations and test the classifier.

In [ ]:
iterations = 200
eta = 0.01
debug = False
w = np.array([[1.0, 1.0, 1.0]]) # Weight vector: X, Y, b
plot_data = []

for k in range(iterations):
    # create vector x: should be feature #2, feature #1, y-intercept or feature #0
    # 1. add the feature #0 to the X vector
    b = np.ones((len(data.index), 1), dtype=float)
    x = np.hstack([data.values[:,:2], b])
    if debug:
        if k + 1 == iterations:
            x

    # 2. create the *labels* vector. the class vector and compute its transpose
    labels = np.array([data[2].to_numpy()])
    labels = labels.T
    if debug:
        if k + 1 == iterations:
            labels

    # 4. compute the dot product of x.w
    # x @ w.T

    # 5. multiply by y
    # y * (x @ w.T)

    # 6. nudge vector w
    # select instances that evaluate to True (Wrongly classified) and add them 
    # to a new array. the line: ((y * (x @ w.T)) < 0).reshape(x.shape[0]) 
    # creates a 1D array of boolean values which is used as row evaluation 
    # subscript in x. only rows that evaluate to TRUE will be added to the xx array.
    xx = x[((labels * (x @ w.T)) < 0).reshape(x.shape[0]), :]
    yy = labels[((labels * (x @ w.T)) < 0).reshape(x.shape[0]), :]
    # debug
    if debug:
        if k + 1 == iterations:
            ((labels * (x @ w.T)) < 0)
            xx
            yy
        
    z = np.sum(yy * xx, axis=0, keepdims=True)
    if debug:
        if k + 1 == iterations:
            z
    
    if debug:
        if k + 1 == iterations:
            nyx = ((yy * xx) * eta)
            nyx
    
    w = w + (z * eta)
    if debug:
        if k + 1 == iterations:
            w
            
    # 7. Append to plot data.
    plot_data.append(w)

# w

## Animate the Weight Vector (2D)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

_ = ax.set_xlabel('Earning:Expenses')
_ = ax.set_ylabel('Late Payments')

_ = ax.scatter(x=data[0],
               y=data[1],
               c=data[2],
               cmap=mpl.colors.ListedColormap(['red', 'blue']),
               marker='o',
               s=40)

line, = ax.plot([], [], c= 'black')
annotations = []

def init():
    line.set_data([], [])
    return line,

def animate(i, ax):    
    A = plot_data[i][0][0].item()
    B = plot_data[i][0][1].item()
    C = plot_data[i][0][2].item()
        
    temp_x_data = np.linspace(0.0, 2.0, num=10)
    temp_y_data = [(-((A * k) + (C)) / B) for k in temp_x_data]
    line.set_data(temp_x_data, temp_y_data)
    
    _ = ax.legend(['Iteration {}'.format(i + 1)])
    
    for k in annotations:
        k.remove()
    annotations[:] = []
    annotations.append(ax.annotate(
        'A:{:.2f} B:{:.2f} C:{:.2f}'.format(A, B, C),
        xy=(10, 230),
        xycoords='axes points',
        bbox=dict(boxstyle='round', fc='w')))
    
    return line,

anim = ani.FuncAnimation(fig,
                         animate,
                         init_func=init,
                         frames=iterations, 
                         fargs=(ax,))
plt.close()
HTML(anim.to_jshtml())

## Plot the Final Line (2D)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

_ = ax.scatter(
    x=data[0], 
    y=data[1], 
    c=data[2], 
    cmap=mpl.colors.ListedColormap(['red', 'blue']),
    marker='o',
    s=40)

A = w[0][0].item()
B = w[0][1].item()
C = w[0][2].item()

temp_x_data = np.linspace(0.0, 2.0, num=10)
temp_y_data = [(-((A * k) + (C)) / B) for k in temp_x_data]

_ = plt.plot(temp_x_data, temp_y_data, c='black')

_ = ax.set_xlabel('Earning:Expenses')
_ = ax.set_ylabel('Late Payments')

## Plot the Final Plane (3D)

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.gca(projection='3d')
frames = 360 * 2
max_elev = 60
min_elev = 15
ele = list(range(min_elev, max_elev, 1))

for k in range(1, int(frames / (max_elev - min_elev))):
    if k % 2 != 0:
        ele += list(range(max_elev, min_elev, -1))
    else:
        ele += list(range(min_elev, max_elev, 1))

def init():
    _ = ax.set_xlabel('Earning:Expenses')
    _ = ax.set_ylabel('Late Payments')
    _ = ax.set_zlabel('Class')
    _ = ax.view_init(azim=0, elev=0)
    
    _ = ax.scatter3D(xs=data[0],
                     ys=data[1],
                     zs=data[2],
                     c=data[2],
                     cmap=mpl.colors.ListedColormap(['red', 'blue']),
                     marker='o',
                     s=40)

    x_surf, y_surf = np.meshgrid(np.linspace(0.0, 10.0, num=20), np.linspace(0.0, 10.0, num=20))

    A = w[0][0].item()
    B = w[0][1].item()
    C = w[0][2].item()
    
    # Find the value of Z using the equation of the plane.
    z  = A * x_surf + B * y_surf + C
    
    _ = ax.plot_surface(x_surf, y_surf, z, color='black', alpha=0.2, rstride=2, cstride=2)
    
    return ax,

def animate(i, ax):
    ax.view_init(elev=ele[i], azim=i)
    
    return ax,

anim = ani.FuncAnimation(fig,
                         animate,
                         init_func=init,
                         frames=frames,
                         interval=50,
                         fargs=(ax,))
plt.close()
HTML(anim.to_jshtml())

***
# End